In [ ]:
const SubgraphResults = ({ results, wrapInHtml }) => {
  const styles = useStyles();
  const [selectedTab, setSelectedTab] = useState("0");

  if (!results || typeof results !== 'object' || Object.keys(results).length === 0) {
    return null;
  }

  // If there's only one query with one subgraph, don't display anything
  if (Object.keys(results).length === 1 && Object.keys(Object.values(results)[0]).length === 1) {
    return null;
  }

  return (
    <div className={styles.container}>
      {Object.entries(results).map(([query, subgraphs], index) => (
        <Dialog key={index}>
          <DialogTrigger disableButtonEnhancement>
            <Card className={styles.queryCard}>
              <CardHeader header={<div className={styles.queryText}>{query}</div>} />
            </Card>
          </DialogTrigger>
          <DialogSurface className={styles.dialogSurface}>
            <DialogBody className={styles.dialogBody}>
              <DialogTitle>
                <div className={styles.dialogTitle}>{query}</div>
              </DialogTitle>
              <DialogContent className={styles.dialogContent}>
                {Object.keys(subgraphs).length > 1 ? (
                  <>
                    <TabList 
                      selectedValue={selectedTab} 
                      onTabSelect={(_, data) => setSelectedTab(data.value)}
                      className={styles.tabList}
                    >
                      {Object.keys(subgraphs).map((subgraphName, idx) => (
                        <Tab key={idx} value={idx.toString()}>
                          {subgraphNameMap[subgraphName] || subgraphName}
                        </Tab>
                      ))}
                    </TabList>
                    <div className={styles.tabContent}>
                      {Object.entries(subgraphs).map(([subgraphName, result], idx) => (
                        selectedTab === idx.toString() && (
                          <div key={idx} className={styles.resultContainer}>
                            <div dangerouslySetInnerHTML={{ __html: wrapInHtml(result) }} />
                          </div>
                        )
                      ))}
                    </div>
                  </>
                ) : (
                  <div className={styles.singleSubgraphContent}>
                    <div dangerouslySetInnerHTML={{ __html: wrapInHtml(Object.values(subgraphs)[0]) }} />
                  </div>
                )}
              </DialogContent>
              <DialogActions className={styles.dialogActions}>
                <DialogTrigger disableButtonEnhancement>
                  <Button appearance="secondary" icon={<Dismiss24Regular />}>
                    Close
                  </Button>
                </DialogTrigger>
              </DialogActions>
            </DialogBody>
          </DialogSurface>
        </Dialog>
      ))}
    </div>
  );
};

export default SubgraphResults;